In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
#input names of csv files created from web scrape
csv_name = 'imdb_2018.csv'
csv2_name = 'imdb_2019.csv'
#input name of json to create
json_name = 'imdb.json'

# Importing Data from Scrape

In [51]:
#import csv 1 to data frame
df1 = pd.read_csv(csv_name, index_col='Unnamed: 0')

In [52]:
#import csv 2 to data frame
df2 = pd.read_csv(csv2_name, index_col='Unnamed: 0')

In [53]:
#concatonate 2 data frames
df = pd.concat([df1, df2],ignore_index=True)

In [54]:
df.head()

,certificate,genre,gross,id,imdb_rating,metascore,runtime,title,votes,year
0,PG,"['Animation', 'Action', 'Adventure']",189936471.0,tt4633694,8.5,87.0,117 min,Spider-Man: Into the Spider-Verse,160975.0,['2018']
1,PG-13,"['Action', 'Adventure', 'Fantasy']",334826690.0,tt1477834,7.2,55.0,143 min,Aquaman,211622.0,['2018']
2,R,"['Crime', 'Drama', 'Thriller']",103804407.0,tt7959026,7.1,58.0,116 min,The Mule,32910.0,['2018']
3,PG-13,"['Biography', 'Comedy', 'Drama']",84068561.0,tt6966692,8.3,69.0,130 min,Green Book,158958.0,['2018']
4,PG-13,"['Biography', 'Drama', 'Music']",215983200.0,tt1727824,8.1,49.0,134 min,Bohemian Rhapsody,311167.0,['2018']


# Cleaning Data

In [55]:
#genre is a string of a list. This will make it a list
df['genre'] = [x.strip('[]').strip("''").replace("', '"," ").split(" ") if type(x)==str else 'NaN' for x in df['genre']]


In [56]:
#runtimes are strings. removing 'min ' and making them floats
df['runtime'] = [float(x.replace(' min','')) if type(x)==str else float('NaN') for x in df['runtime']] 

In [59]:
#year is a string of a list.  removing the brackets and extra quote marks
df['year'] = [x.strip("['']") if type(x)==str else float('NaN') for x in df['year']]

In [60]:
df.head()

,certificate,genre,gross,id,imdb_rating,metascore,runtime,title,votes,year
0,PG,"[Animation, Action, Adventure]",189936471.0,tt4633694,8.5,87.0,117.0,Spider-Man: Into the Spider-Verse,160975.0,2018
1,PG-13,"[Action, Adventure, Fantasy]",334826690.0,tt1477834,7.2,55.0,143.0,Aquaman,211622.0,2018
2,R,"[Crime, Drama, Thriller]",103804407.0,tt7959026,7.1,58.0,116.0,The Mule,32910.0,2018
3,PG-13,"[Biography, Comedy, Drama]",84068561.0,tt6966692,8.3,69.0,130.0,Green Book,158958.0,2018
4,PG-13,"[Biography, Drama, Music]",215983200.0,tt1727824,8.1,49.0,134.0,Bohemian Rhapsody,311167.0,2018


# Creating SQL Database 

In [61]:
import sqlite3

In [62]:
#setting up connection to movies data base and cursor
conn = sqlite3.connect('movies.db')
c = conn.cursor()

In [63]:
import json

In [64]:
#converting df to json file
df.to_json(json_name, 'records')

In [65]:
#importing json file
with open(json_name) as json_file:  
    data = json.load(json_file)

In [69]:
#setting up query to create movies table
create_query = """CREATE TABLE IF NOT EXISTS movies
(imdb_id TEXT PRIMARY KEY, title TEXT, certificate TEXT, gross REAL, imdb_rating REAL, metascore REAL, runtime REAL, votes INTEGER, year INTEGER)"""


In [70]:
#function to create SQLite table. Intakes create_query 
def create_table(query):
    c.execute(query)

#function to close connection
def close_c_conn():
    c.close()
    conn.close()

In [74]:
#function to enter data (json file) into the movies SQL table
#data is entered one row at a time
def movie_data_entry(data):
 
    for movie in data:
        imdb_id = movie['id']
        cert = movie['certificate']
        gross = movie['gross']
        rate = movie['imdb_rating']
        score = movie['metascore']
        runtime = movie['runtime']
        title = movie['title']
        votes = movie['votes']
        year = movie['year']
        
        c.execute("INSERT INTO movies (imdb_id, certificate, gross, imdb_rating, metascore, runtime, title, votes, year) VALUES (?,?,?,?,?,?,?,?,?)",
          (imdb_id, cert, gross, rate, score, runtime,title,votes,year))
        
       
        
    conn.commit()

In [72]:
create_table(create_query)

In [75]:
movie_data_entry(data)

In [76]:
pd.read_sql_query("select * from movies limit 10;", conn)

,imdb_id,title,certificate,gross,imdb_rating,metascore,runtime,votes,year
0,tt4633694,Spider-Man: Into the Spider-Verse,PG,189936471.0,8.5,87.0,117.0,160975,2018
1,tt1477834,Aquaman,PG-13,334826690.0,7.2,55.0,143.0,211622,2018
2,tt7959026,The Mule,R,103804407.0,7.1,58.0,116.0,32910,2018
3,tt6966692,Green Book,PG-13,84068561.0,8.3,69.0,130.0,158958,2018
4,tt1727824,Bohemian Rhapsody,PG-13,215983200.0,8.1,49.0,134.0,311167,2018
5,tt4123430,Fantastic Beasts: The Crimes of Grindelwald,PG-13,159555901.0,6.7,52.0,134.0,144529,2018
6,tt4154756,Avengers: Infinity War,PG-13,678815482.0,8.5,68.0,149.0,609674,2018
7,tt4701182,Bumblebee,PG-13,127195589.0,7.0,66.0,114.0,66418,2018
8,tt1517451,A Star Is Born,R,215252091.0,7.8,88.0,136.0,223942,2018
9,tt6491178,Dragged Across Concrete,R,NaN,7.4,63.0,159.0,4566,2018


In [77]:
#closing connection to movies db
close_c_conn()